# 0. Prerequisits

1. Make sure to install [SeisMIC](https://github.com/PeterMakus/SeisMIC) version 0.5.14 or later. Currently, this version is only available over the GitHub `dev` branch. Follow the instructions provided [here](https://petermakus.github.io/SeisMIC/modules/get_started.html#via-github) to install SeisMIC's developer version.
2. Compute Cross-Correlations between (a) the stations that you suspect to have a clock shift and stations that are operating normally. Follow the instructions in [SeisMIC's tutorial](https://petermakus.github.io/SeisMIC/modules/tutorials.html) to do so. For the example at MSH, we will want to have data between approximately May 2013 and February 2014 and data from the stations *UW.EDM*, *UW.FL2*, *UW.HSR*, *UW.SHW*, *UW.JUN*, *UW.SOS*, *UW.YEL* (the stations with  a clock shift) and some stable stations, I used: *CC.VALT*, *PB.B202*, and *PB.B204*. Note that you will not have to compute the cross-correlations between the UW stations as they are all digitised over a common clock (i.e., no clock shift will be visible).

# 1. Compute clock shift for each station pair.

You will have to adapt the parameters. The code is still fairly exemplary. Read the source code, there are some options that you might want to change. I gave some suggestions

In [ ]:
# Import some necessary modules
import os
import glob
import fnmatch
from copy import deepcopy

from obspy import UTCDateTime
import numpy as np

from seismic.db.corr_hdf5 import CorrelationDataBase as CorrDB


In [ ]:
# Define some functions for below

def check_data_availability(infile, time_to_check: UTCDateTime):
    """
    Check if the data is available inside the h5 file. Also check what
    channel combinations are there
    """
    netcode, statcode = os.path.basename(infile).split('.')[:2]
    with CorrDB(infile, mode='r') as cdb:
        chans = cdb.get_available_channels(
            'subdivision', netcode, statcode)
        av_starts = cdb.get_available_starttimes(
            netcode, statcode, 'subdivision', chans[0]
        )
        av_starts = np.array([UTCDateTime(x) for x in av_starts[chans[0]]])
        if min(abs(av_starts - time_to_check)) > 86400:
            print(f'skipping {infile} because of missing data')
            return []
    return chans


def compute_clock_drift_for_cha(infile, chacode, starttime, endtime, tw):
    """
    Compute the clock drift for a single channel
    """
    netcode, statcode = os.path.basename(infile).split('.')[:2]
    with CorrDB(infile, mode='r') as cdb:
        try:
            cst = cdb.get_data(
                netcode, statcode, chacode, 'subdivision')
            if cst.count() == 0:
                raise IndexError
        except (KeyError, IndexError):
            print(
                f'For {netcode}.{statcode}.{chacode} no data is '
                f'available between {starttime} and {endtime}'
            )
            return False
    try:
        cb = cst.create_corr_bulk(
            inplace=True, times=[starttime, endtime], channel=chacode)
    except ValueError as e:
        print(e)
        return False

    # Option 1: Use the first 90 days as reference. To adjust 0 properly
    # And resample the
    # Correlation Functions evenly. This will help you to accurately locate
    # the drop
    starttimes_new = np.array([starttime + dt for dt in np.arange(
        (endtime-starttime)//3600)*3600]) 
    endtimes_new = starttimes_new + 3600
    cb = cb.resample(starttimes_new, endtimes_new)
    ref_trc = cb[:90*24].extract_trace()
    cb.smooth(24*10)
    
    # option 2: Resample the Correlation Functions to only two functions.
    # One before and one after the drop occurs. Then compare the two
    # If you use this one, you need to comment out the above lines

    # cb = cb.resample(
    #     np.array([starttime, UTCDateTime(2013, 10, 3)]),
    #     np.array([UTCDateTime(2013, 10, 25), endtime]))
    # ref_trc=cb[0]
    
    # You can adjust the parameters. For instance shift range=1 checks
    # for shifts of maxmimum 1s in either direction
    dt = cb.measure_shift(
        shift_range=1, shift_steps=201, return_sim_mat=True,
        tw=tw, ref_trc=ref_trc)
    return dt
    

In [ ]:


# ## Define Parameters. You will have to change things here


# Time window to invert the shifts for
# We can use several time windows and jointly invert to raise the precision /
# to avoid velocity changes being mistaken as clock shifts
# Each time window should encompass several periods of your dominant frequency
tw = [[tws, twe] for tws, twe in zip(np.arange(9)*5 + 5, np.arange(9)*5 + 20)]

starttime = UTCDateTime(2013, 5, 1)
endtime = UTCDateTime(2014, 2, 1)

droptime = UTCDateTime(2013, 9, 3)

infolder = '/This/is/where/the/correlation/files/are'

outfolder = '/where/the/output/will/be/saved'

## The actual code. This could be parallised using mpi4py

os.makedirs(outfolder, exist_ok=True)

for infile in glob.glob(os.path.join(infolder, '*.h5')):
    netcode, statcode = os.path.basename(infile).split('.')[:2]
    chans = check_data_availability(infile, droptime)
    for chacode in chans:
        print(
            f'working on {infile} for {netcode}.{statcode}.{chacode}')
        dt = compute_clock_drift_for_cha(
            infile, chacode, starttime, endtime, tw)
        dt.save(os.path.join(outfolder, f'DT-{dt.stats.id}.npz'))

# 2. Stack and plot your shift
In this very particular case, we are fortunate enough that we have a few stations that exhibit exactly the same shift, so we can stack the results to reduce errors. the code below shows you how to do that

In [ ]:
from seismic.monitor.dv import read_dv
from seismic.monitor.monitor import average_components

# averaging is pretty easy

# read all the shift files
shifts = read_dv(outfolder, 'DT-*.npz')

# average them. Under the hood, this stacks the resulting matrix from the
# grid search and computes the new maximum
avg_shifts = average_components(shifts, save_scatter=False)


# ## Plot the results
avg_shifts.plot()